# Coberturas vacinais por município

A planilha de cobertura vacinal, obtida do site do [Minitério da Saúde](https://infoms.saude.gov.br/extensions/SEIDIGI_DEMAS_VACINACAO_CALENDARIO_NACIONAL_MENU_COBERTURA/SEIDIGI_DEMAS_VACINACAO_CALENDARIO_NACIONAL_MENU_COBERTURA.html),  apresenta células unidas, linhas com totais, hífen para representar ausência de dados e identificação do município associada ao código do IBGE com seis dígitos como apresentado a seguir.

<figure>
  <img
  src="Figuras/PlanilhaCoberturas.png"
  alt="Planilha de coberturas vacinais">
  <figcaption>Planilha de coberturas vacinais</figcaption>
</figure>



O objetivo é gerar uma planilha com as coberturas vacinais para os diferentes imunobiológicos com colunas separadas para o nome do município e cógido do IBGE. Esta última será utilizada para unir esta planilha com a malha municipal do Brasil obtida no site do [IBGE](https://www.ibge.gov.br/geociencias/downloads-geociencias.html) e simplificada utilizando-se o [mapshaper](https://github.com/mbloch/mapshaper) reduzindo-se a quantidade de pontos para 1%, tornando o arquivo menor sem prejuízo para visualização nas escalas utilizadas.

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
coberturas = pd.read_excel('../data/coberturas_2024.xlsx')

In [3]:
coberturas.reset_index(inplace=True)

In [4]:
for coluna in ['Região Ocorrência', 'UF Residência', 'Macrorregião Saúde', 'Região de Saúde', 'Município Residência']:
    coberturas.drop(coberturas.loc[coberturas[coluna]=='Totais'].index, axis=0, inplace=True)

In [5]:
coberturas.replace({'-': None}, inplace=True)

In [6]:
coberturas['Código IBGE 6'] = coberturas['Município Residência'].apply(lambda x: x.split('-')[0].strip())

In [7]:
coberturas['Nome município'] = coberturas['Município Residência'].apply(lambda x: x.split('-')[1].strip())

In [8]:
coberturas.drop(['index', 
                 ' ', 
                 'Região Ocorrência', 
                 'Macrorregião Saúde', 
                 'Região de Saúde', 
                 'Município Residência',
                 'Imunobiológico'], 
                axis=1, 
                inplace=True)

## Gera arquivo cvs de coberturas vacinais com geometria

Optou-se por exportar para cvs pois os arquivos do tipo ESRI Shapefile tem limitação de 10 caracteres no máximo para o nome de cada coluna. A exportação para geopackage gerou um arquivo com pequenos problemas (parece estar associado ao GDAL). Como existe uma coluna com geometria em WKT, o formato csv atende ao objetivo.

In [9]:
municipios = gpd.read_file('../data/BR_Municipios_2022/BR_Municipios_2022.shp', encoding='utf-8')

In [10]:
municipios['CD_MUN_6'] = municipios['CD_MUN'].apply(lambda x: x[0:6])

In [11]:
municipios_cobertura = municipios.merge(coberturas, how='left', left_on='CD_MUN_6', right_on='Código IBGE 6')

In [12]:
municipios_cobertura.drop(['AREA_KM2', 'CD_MUN_6', 'UF Residência', 'Código IBGE 6', 'Nome município'], axis=1, inplace=True)

In [13]:
imunogenos =  ['BCG', 
               'DTP', 
               'DTP (1° Reforço)',
               'dTpa Adulto', 
               'Febre Amarela',
               'Hepatite A Infantil', 
               'Hepatite B', 
               'Hepatite B (< 30 Dias)',
               'Meningo C', 
               'Meningo C (1° Reforço)', 
               'Penta (DTP/HepB/Hib)',
               'Pneumo 10', 
               'Pneumo 10 (1° Reforço)', 
               'Polio Injetável (VIP)',
               'Polio Oral Bivalente', 
               'Rotavírus', 
               'Tríplice Viral - 1° Dose',
               'Tríplice Viral - 2° Dose',
               'Varicela']

In [14]:
for imunogeno in imunogenos:
    municipios_cobertura[imunogeno] = municipios_cobertura[imunogeno].astype(float)

In [15]:
municipios_cobertura.to_csv('../data/municipios_coberturas.csv', encoding='utf-8')